# Taller 1 Visualización de Datos

**Integrante:** Christian Pérez Flores

**Profesora:** Ana Moya Beltrán

**Ayudante:** Diego Santibáñez


# Bibliotecas y conexión a API de Reddit

En primer lugar se instalan e importan las bibliotecas necesarias, utilizando PRAW para conectarnos a la API de Reddit gracias a las credenciales obtenidas a través de [Reddit Apps](https://www.reddit.com/prefs/apps/)

In [ ]:
!pip install duckdb

In [ ]:
!pip install praw

In [ ]:
import pandas as pd
import numpy as np
import praw
import datetime

In [ ]:
reddit = praw.Reddit(
     client_id="M04shejV766NFiwJlZ-QIQ",
     client_secret="THwpb6fViqOf9biT1VoQo3vSNtNDkg",
     user_agent="Fun_School8641",
     check_for_async=False
)

# Extracción de datos

Se eligen las comunidades a explorar y se organiza la estructura que recibirá la información extraida, en este caso se optó por transformar los datos desde dos dataframes distintos (posts y comentarios) a dos archivos CSV distintos, cada uno con sus nombres respectivos para no generar confusiones en trabajos futuros.


## Elección de comunidades

Las comunidades elegidas buscan tener enfoques distintos, que permitan generar análisis y gráficos variados y diferenciados entre sí, se eligieron tres subreddits con comportamientos y características distintas.

1.   **SplatoonMeta:** Comunidad de discusión para el videojuego Splatoon, en ella se habla sobre estrategias e información referida al mismo en un contexto de competitividad. Se eligió esta comunidad debido a la variedad de elementos relacionadas al videojuego que se mencionan (nombres de distintas armas, roles de miembros de un equipo, nombres de estrategias y composiciones de equipo, distintos torneos, entre otros), lo que podría permitir generar gráficos variados, además de su comportamiento enfocado en la colaboración estratégica, el análisis técnico y el intercambio de información de alto nivel, lo que responde a su naturaleza competitiva y orientada a los e-sports, los miembros de esta comunidad no participan solo como jugadores casuales, sino como analistas, estrategas y competidores activos.

In [ ]:
SplatoonMeta = "SplatoonMeta"

2.   **LearnPython**: Comunidad dedicada al aprendizaje del lenguaje de programación Python, orientada principalmente a principiantes. En ella se comparten dudas, recursos, ejemplos de código y consejos prácticos. Se eligió esta comunidad debido a la variedad de temas relacionados al proceso de aprendizaje que se mencionan, coo errores comunes, recomendaciones de cursos, preguntas frecuentes o aplicaciones básicas. Además, presenta un comportamiento centrado en la colaboración, la resolución colectiva de problemas y el apoyo entre aprendices, lo que responde a su naturaleza formativa y accesible.

In [ ]:
LearnPython = "learnpython"

3.   **AskBaking**: Comunidad enfocada a resolver dudas y compartir consejos sobre repostería, donde los miembros buscan mejorar sus técnicas, experimentar con nuevas recetas o solucionar problemas comunes al hornear. Se eligió esta comunidad debido a la variedad de interacciones que se dan, como preguntas específicas sobre ingredientes, técnicas de decoración, consejos prácticos y recomendaciones de productos, lo que podría permitir generar análisis sobre las tendencias de preguntas más frecuentes y los temas más discutidos. Además, el comportamiento de la comunidad se centra más en el intercambio informal de experiencias y el apoyo mutuo, con un ambiente más relajado y colaborativo orientado al disfrute personal y el aprendizaje a través de la experimentación.

In [ ]:
AskBaking = "askbaking"

## Estructuración y extracción de resultados

En primer lugar se visualizó información básica y pública de las comunidades elegidas, principalmente para verificar que existiera una cantidad adecueda de suscriptores y una fecha de creación razonable, para así asegurar tener una cantidad de publicaciones y comentarios necesaria (al menos 100 publicaciones con al menos 10 comentarios cada una).

In [ ]:
def info_subreddit(subreddit):
  print(f"Nombre completo: r/{subreddit.display_name}")
  print(f"Titulo: {subreddit.title}")
  print(f"Descripción: {subreddit.public_description}")
  print(f"Suscriptores: {subreddit.subscribers}")
  print(f"Usuarios activos: {subreddit.accounts_active}")
  print(f"Fecha de creación: {datetime.datetime.fromtimestamp(subreddit.created_utc).strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
info_subreddit(reddit.subreddit(SplatoonMeta))

Nombre completo: r/SplatoonMeta
Titulo: Competitive Splatoon Discussion
Descripción: This sub is for discussion of Splatoon as a competitive game, including its metagame, strategies, and esports-related information. Feel free to ask for help and advice for improving your skills.  Please use /r/Splatoon or /r/Splatoon_3 for general Splatoon posts.
Suscriptores: 9776
Usuarios activos: 5
Fecha de creación: 2015-05-29 09:00:10


In [ ]:
info_subreddit(reddit.subreddit(LearnPython))

Nombre completo: r/learnpython
Titulo: Python Education
Descripción: Subreddit for posting questions and asking for general advice about all topics related to learning python.
Suscriptores: 918644
Usuarios activos: 127
Fecha de creación: 2009-10-02 15:59:41


In [ ]:
info_subreddit(reddit.subreddit(AskBaking))

Nombre completo: r/askbaking
Titulo: Ask Baking
Descripción: Welcome to /r/AskBaking! This subreddit is devoted to the discussion of questions related to baking, the process, and requests for help on your results!

Please read the rules; posts will be deleted if they do not follow our rules.
Suscriptores: 289602
Usuarios activos: 35
Fecha de creación: 2016-06-16 19:01:43


Se definió una función que permitiera organizar los datos obtenidos tras la extracción con Web Scrapping.

Para las publicaciones a extraer, se guardan:
- ID del post  
- Título  
- Fecha de creación  (con formato año-mes-día y hora-minuto-segundo)
- Autor (en caso de que el autor ya no exista, se le nombra como "ELIMINADO")
- URL  
- Número de comentarios  
- Texto del post
- Tiene texto (para determinar si la publicación tiene o no texto)
- Puntaje

Mientras que para los comentarios de las publicaciones extraidas, se guardan:
- ID del post
- ID del comentario  
- Autor  
- Texto
- Fecha


In [ ]:
def extraccion_info_post(post):
  fecha = datetime.datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
  autor = str(post.author) if post.author else "ELIMINADO"
  tiene_texto = "Sí" if post.selftext else "No"

  datos['id_post'].append(post.id)
  datos['titulo_post'].append(post.title)
  datos['fecha_post'].append(fecha)
  datos['autor_post'].append(autor)
  datos['URL'].append(post.url)
  datos['numero_comentarios'].append(post.num_comments)
  datos['texto_post'].append(post.selftext)
  datos['tiene_texto'].append(tiene_texto)
  datos['score'].append(post.score)

  post.comments.replace_more(limit=0) #Evita cargar "load more comments"
  for comment in post.comments[:10]:
    if not comment.stickied:
      comentarios['id_post'].append(post.id)
      comentarios['id_comentario'].append(comment.id)
      comentarios['autor_comentario'].append(str(comment.author) if comment.author else "ELIMINADO")
      comentarios['texto_comentario'].append(comment.body)
      comentarios['fecha_comentario'].append(datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'))

Para la extracción de las publicaciones se crea la estructura necesaria para almacenar los datos requeridos según lo explicado con anterioridad. En este caso se limitó el máximo número de publicaciones a 150 por comodidad pero podría aumentarse en un futuro de ser necesario, se consideran publicaciones válidas a aquellas con un mínimo de 10 comentarios cada una.

Una vez que los datos son extraídos se separan en dos dataframes distintos (posts y comentarios), los que posteriormente se transforman en un archivo CSV para trabajos futuros.

In [ ]:
def extraer_posts_validos(sub, max_posts=150, min_comentarios=10):
    global datos, comentarios  # <-- Permite que la estructura a crear sea global y se pueda usar en las distintas funciones.

    # Creación y reinicio de la estructura para empezar "limpio" luego de la primera extraacción,
    # de otro modo las siguientes almacenarían los resultados de las comunidades anteriores.

    datos = {
        'id_post': [],
        'titulo_post': [],
        'fecha_post': [],
        'autor_post': [],
        'URL': [],
        'numero_comentarios': [],
        'texto_post': [],
        'tiene_texto': [],
        'score': []
    }

    comentarios = {
        'id_post': [],
        'id_comentario': [],
        'autor_comentario': [],
        'texto_comentario': [],
        'fecha_comentario': []
    }

    posts_almacenados = set()
    contador_validos = 0 #Permitirá verificar que exista la cantidad necesaria de publicaciones

    for submission in sub.top(limit=None, time_filter='all'):
        if submission.id in posts_almacenados: #Evita que hayan publicaciones duplicadas
            continue
        if submission.num_comments < min_comentarios: #Se salta las publicaciones con menos de 10 comentarios
            continue

        extraccion_info_post(submission)
        posts_almacenados.add(submission.id)
        contador_validos += 1

        if contador_validos >= max_posts and contador_validos >= 100:
            break  # Salimos del for aquí, y el guardado se hace después

    if contador_validos >= 100: #Se asegura que haya al menos la cantidad requerida de publicaciones antes de guardar en los respectivos dataframes
        df_posts = pd.DataFrame(datos)
        df_comentarios = pd.DataFrame(comentarios)
        nombre_sub = sub.display_name.lower()
        df_posts.to_csv(f'posts_{nombre_sub}.csv', index=False, encoding='utf-8-sig') #Se genera el CSV para almacenar publicaciones
        df_comentarios.to_csv(f'comentarios_{nombre_sub}.csv', index=False, encoding='utf-8-sig') #Se genera el CSV para almacenar comentarios
        print(f"Número de posts válidos almacenados: {contador_validos}")
    else: #En caso de no haberse guardado 100 posts como mínimo, entonces se manda un mensaje en pantalla
        print(f"Número de posts válidos almacenados: {contador_validos}, no se ha generado ningún archivo CSV")

Se llaman a las funciones creadas para la extracción de los resultados, pasándole el nombre de las comunidades previamente elegidas, lo que generará los CSV respectivos.

In [ ]:
sub = reddit.subreddit(SplatoonMeta)
extraer_posts_validos(sub)

Número de posts válidos almacenados: 150


In [ ]:
sub = reddit.subreddit(LearnPython)
extraer_posts_validos(sub)

Número de posts válidos almacenados: 150


In [ ]:
sub = reddit.subreddit(AskBaking)
extraer_posts_validos(sub)

Número de posts válidos almacenados: 150
